# Data quality for linkurious

In [11]:
sourcepath = 'C:/_PYDAI/1-Data/'
neo4jimportpath='C:/_PYDAI/2-Neo4j_db/duns.graphdb/import/'
os.chdir(sourcepath)
outpath='C:/_PYDAI/8-PythonProjects/uploadinneo4j/'

In [12]:
filename__duns_node='DunsID_normalized.csv'
filename_duns_edge='DunsRelations_normalized.csv'
filename__erp_node='ErpID_normalized.csv'
filename_erp_edge='ErpRelations_normalized.csv'

In [13]:
script_duns_node='upload_duns_node.cql'
script_duns_edge='upload_duns_edge.cql'
script_erp_node='upload_erp_node.cql'
script_erp_edge='upload_erp_edge.cql'

# Write upload neo4j queries

# For nodes

### Function

In [14]:
def createlineuploadnode(colname,nodename,neo4jname=None):
    colname =str(colname).lower().lstrip().rstrip().replace(' ','')
    if neo4jname is None:
        neo4jname = colname
    s="{0}.{1} = {2}.{3}".format(nodename,neo4jname,'csvLine',neo4jname)
    return s
def createfullquery(filename,path=neo4jimportpath,labelname='DunsID',nodeid='duns'):
    s='''
    USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM 'file:///{0}' AS csvLine
    MERGE (n:{labelname}{{ {nodeid}: csvLine.{nodeid} }})
    SET
    '''
    s=s.format(filename,labelname=labelname,nodeid=nodeid)
    mycols=pd.read_csv(path+filename,nrows=5).columns
    for c in mycols:
        r = createlineuploadnode(c,nodename='n')
        r="\t"+r+",\n"
        s+=r
    s=s.rstrip(',\n') +';'
    return s

### Duns

In [15]:
query = createfullquery(filename=filename__duns_node,path=neo4jimportpath,labelname='DunsID',nodeid='duns')
print(query)
myfile=open(outpath+script_duns_node,'w').close()
print(query,file=open(outpath+script_duns_node,'a'))


    USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM 'file:///DunsID_normalized.csv' AS csvLine
    MERGE (n:DunsID{ duns: csvLine.duns })
    SET
    	n.duns = csvLine.duns,
	n.name = csvLine.name,
	n.businessname = csvLine.businessname,
	n.street = csvLine.street,
	n.city = csvLine.city,
	n.zip = csvLine.zip,
	n.county = csvLine.county,
	n.state = csvLine.state,
	n.state_abbreviation = csvLine.state_abbreviation,
	n.country_name = csvLine.country_name,
	n.country_code = csvLine.country_code,
	n.continent_code = csvLine.continent_code,
	n.commregno = csvLine.commregno,
	n.rank_2016 = csvLine.rank_2016,
	n.source_relation = csvLine.source_relation,
	n.dnb_dp_duns = csvLine.dnb_dp_duns,
	n.dnb_dp_name = csvLine.dnb_dp_name,
	n.dnb_dp_city = csvLine.dnb_dp_city,
	n.dnb_dp_countrycode = csvLine.dnb_dp_countrycode,
	n.dnb_du_duns = csvLine.dnb_du_duns,
	n.dnb_du_name = csvLine.dnb_du_name,
	n.dnb_du_city = csvLine.dnb_du_city,
	n.dnb_du_countrycode = csvLine.dnb_du_countrycode,


### ERP

In [16]:
query = createfullquery(filename=filename__erp_node,path=neo4jimportpath,labelname='ErpID',nodeid='erpid')
print(query)
myfile=open(outpath+script_erp_node,'w').close()
print(query,file=open(outpath+script_erp_node,'a'))


    USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM 'file:///ErpID_normalized.csv' AS csvLine
    MERGE (n:ErpID{ erpid: csvLine.erpid })
    SET
    	n.erp_system_id = csvLine.erp_system_id,
	n.erp_location_id = csvLine.erp_location_id,
	n.erp_supplier_id = csvLine.erp_supplier_id,
	n.erp_name = csvLine.erp_name,
	n.erp_street = csvLine.erp_street,
	n.erp_zip = csvLine.erp_zip,
	n.erp_city = csvLine.erp_city,
	n.erp_country_name = csvLine.erp_country_name,
	n.erp_tax_no = csvLine.erp_tax_no,
	n.erp_commregno = csvLine.erp_commregno,
	n.turnover2016 = csvLine.turnover2016,
	n.ordervolume2016 = csvLine.ordervolume2016,
	n.turnover2017 = csvLine.turnover2017,
	n.ordervolume2017 = csvLine.ordervolume2017,
	n.data_status = csvLine.data_status,
	n.erpid = csvLine.erpid;


## Relations

In [17]:
s='''
MATCH (a:DunsID),(b:DunsID)
WHERE b.dunsid = a.parentdunsid and not a.parentdunsid=a.dunsid
MERGE (a)-[r:isChildOf]->(b)
WITH a,r
SET r.source_relation = a.source_relation
'''
print(s)


MATCH (a:DunsID),(b:DunsID)
WHERE b.dunsid = a.parentdunsid and not a.parentdunsid=a.dunsid
MERGE (a)-[r:isChildOf]->(b)
WITH a,r
SET r.source_relation = a.source_relation



In [18]:
def createedgequery(filename,path=neo4jimportpath,filerelation='isChildOf',sourcelabel='DunsID',targetlabel='DunsID',
                    neo4jsourceid='duns',neo4jtargetid='duns'):
    df=pd.read_csv(path+filename,nrows=5)
    mycols=df.columns
    filesourceid=mycols[0]
    filetargetid=mycols[2]
    p='''USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM 'file:///{filename}' AS csvLine
    MERGE (a:{sourcelabel}{{ {neo4jsourceid}: csvLine.{filesourceid} }})-[r:csvLine.{filerelation}]->(b:{targetlabel}{{ {neo4jtargetid}: csvLine.{filetargetid} }})
    WITH r
    SET
    '''
    p=p.format(filename=filename,
        sourcelabel=sourcelabel,neo4jsourceid=neo4jsourceid,
             filesourceid=filesourceid,filerelation='filerelation',
             targetlabel=targetlabel,neo4jtargetid=neo4jtargetid,filetargetid=filetargetid)
    for c in mycols:
        r = createlineuploadnode(c,nodename='r')
        r="\t"+r+",\n"
        p+=r
    p=p.rstrip(',\n') +';'
    return p

### Duns

In [19]:
query = createedgequery(filename=filename_duns_edge,
                        path=neo4jimportpath,
                       sourcelabel='DunsID',
                       neo4jsourceid='duns',
                        filerelation='isChildOf',
                       targetlabel='DunsID',
                       neo4jtargetid='duns')
print(query)
myfile=open(outpath+script_duns_edge,'w').close()
print(query,file=open(outpath+script_duns_edge,'a'))

USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM 'file:///DunsRelations_normalized.csv' AS csvLine
    MERGE (a:DunsID{ duns: csvLine.source_id })-[r:csvLine.filerelation]->(b:DunsID{ duns: csvLine.target_id })
    WITH r
    SET
    	r.source_id = csvLine.source_id,
	r.edge_label = csvLine.edge_label,
	r.target_id = csvLine.target_id,
	r.levelofsource = csvLine.levelofsource,
	r.source_relation = csvLine.source_relation,
	r.changed_by = csvLine.changed_by,
	r.changed_at = csvLine.changed_at;


### Erp

In [20]:
query = createedgequery(filename=filename_erp_edge,
                        path=neo4jimportpath,
                       sourcelabel='ErpID',
                       neo4jsourceid='erpid',
                        filerelation='isChildOf',
                       targetlabel='DunsID',
                       neo4jtargetid='duns')
print(query)
myfile=open(outpath+script_erp_edge,'w').close()
print(query,file=open(outpath+script_erp_edge,'a'))

USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM 'file:///ErpRelations_normalized.csv' AS csvLine
    MERGE (a:ErpID{ erpid: csvLine.erp_system_id })-[r:csvLine.filerelation]->(b:DunsID{ duns: csvLine.erp_supplier_id })
    WITH r
    SET
    	r.erp_system_id = csvLine.erp_system_id,
	r.erp_location_id = csvLine.erp_location_id,
	r.erp_supplier_id = csvLine.erp_supplier_id,
	r.edge_label = csvLine.edge_label,
	r.source_relation = csvLine.source_relation,
	r.changed_by = csvLine.changed_by,
	r.changed_at = csvLine.changed_at,
	r.target_dunsid = csvLine.target_dunsid,
	r.erpid = csvLine.erpid;
